In [1]:
from Google_Scholar_API import Scraper
import ipywidgets as widgets
from tkinter import filedialog, Tk
from IPython.display import display, Markdown, clear_output

In [2]:
# Checks if a scraper is already running
IS_RUNNING = False

# A global variable to use the folder selector
DOWNLOAD_DIR = ''

# The Scraper
SCRAPER = None

def select_directory():
    root = Tk()
    dirname = filedialog.askdirectory(parent=root,initialdir="/",title='Please select a directory')
    root.destroy()
    if len(dirname)>0 :
        global DOWNLOAD_DIR
        DOWNLOAD_DIR = dirname

In [3]:
# Definition of all widgets

dl_dir = widgets.Button(
    description='Select Download Folder',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Select Download Directory',
    layout={'width': 'max-content'}
)

out = widgets.Output()

def on_dl_dir_clicked(_):
    with out:
        clear_output()
        select_directory()
        print(DOWNLOAD_DIR)

language = widgets.RadioButtons(
    options=['en', 'fr'],
    value='en', # Defaults to 'pineapple'
    layout={'width': 'max-content'}, # If the items' names are long
    disabled=False
)

captcha = widgets.Checkbox(
    value=False,
    description='Captcha Bypass',
    disabled=False,
    indent=False
)

sleep = widgets.IntSlider(
    value=5,
    min=3,
    max=20,
    step=1,
    description='Web loading Time:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    style = {'description_width': 'initial'}
)

pages = widgets.IntRangeSlider(
    value=[1, 20],
    min=1,
    max=100,
    step=1,
    description='Pages to Scrape:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    style = {'description_width': 'initial'}
)

search = widgets.Text(
    placeholder='Search terms..',
    description='🔎',
    disabled=False,
    layout={'width': '90%'}
)

send = widgets.Button(
    description='Download',
    disabled=False,
    button_style='success', #'info', 'warning', 'danger' or ''
    tooltip='Download',
    layout={'width': 'max-content'}
)

out2 = widgets.Output()
def on_send_clicked(_):
    with out2:
        clear_output()
        global SCRAPER
        global IS_RUNNING

        if not IS_RUNNING :
            if DOWNLOAD_DIR == '':
                print('Select a Download Folder !')
                return
            else :
                SCRAPER = Scraper(DOWNLOAD_DIR, language.value, captcha.value, sleep.value)
                IS_RUNNING = True
        if language.value != SCRAPER.language :
            SCRAPER.set_language(language.value)
        if DOWNLOAD_DIR != SCRAPER.download_dir :
            SCRAPER.set_download_dir(DOWNLOAD_DIR)
        if sleep.value != SCRAPER.sleep :
            SCRAPER.set_sleep(sleep.value)
        if captcha.value != SCRAPER.captcha :
            SCRAPER.set_captcha(captcha.value)
            print('Reload the search after having completed the Captchas')
        else :
            SCRAPER.download_files(search.value, pages.value[0], pages.value[1])
        
reset = widgets.Button(
    description='Reset Scraper',
    disabled=False,
    button_style='warning',
    tooltip='reset',
    layout={'width': 'max-content'}
)

def on_reset_clicked(_):
    global SCRAPER
    global IS_RUNNING
    SCRAPER = None
    IS_RUNNING = False
    
    
# Layout to center objects
box_layout = widgets.Layout(display='flex',
                flex_flow='column',
                align_items='center',
                width='max-content')

center_100 = widgets.Layout(display='flex',
                flex_flow='column',
                align_items='center',
                width='90%',
                justify_content='space-between')



In [4]:
# Display portion 

dl_dir.on_click(on_dl_dir_clicked)
send.on_click(on_send_clicked)
reset.on_click(on_reset_clicked)

widgets.VBox(
    children=[
        search,
        widgets.VBox([dl_dir,out]),
        widgets.HBox([
            widgets.VBox([        
                widgets.VBox(children=[widgets.Label('Language of Google Engine:'), language], layout=box_layout),        
                captcha]),
            widgets.VBox([sleep,pages])]),
        send,
        reset,
        out2],
    layout=center_100)